# Intermediate Julia for scientific computing

This workshop is designed to introduce two fundamental concepts in Julia: **types** and **metaprogramming**.

In order to cover various key uses of types in Julia, we have chosen to frame the discussion around a concrete topic in scientific computing, namely **root-finding**. 
The goal is *not* to learn algorithms for root finding *per se*, but rather to have a (pseudo-)real context in which to explore various concepts centered around types and how they arise naturally in real applications of Julia, in particular applications of **multiple dispatch**, which is one of the core choices in Julia that differentiate it from other common languages.

We will implement a couple of root-finding algorithms just to have something to work with. These will just be toy implementations that are far away from the best implementations. 

Instead we should use one of the high-quality packages that are available in Julia for this purpose. The large number of them shows the importance of root finding. The ones that I am aware of are the following (in alphabetical order):

- Single root of a nonlinear function:
    - [`NLsolve.jl`](https://github.com/JuliaNLSolvers/NLsolve.jl)
    - [`Roots.jl`](https://github.com/JuliaMath/Roots.jl)

- All roots of polynomial:
    - [`HomotopyContinuation.jl`](https://www.juliahomotopycontinuation.org)
    - [`PolynomialRoots.jl`](https://github.com/giordano/PolynomialRoots.jl)
    - [`Polynomials.jl`](https://github.com/JuliaMath/Polynomials.jl)
    
- All roots of a nonlinear function:
    - [`ApproxFun.jl`](https://github.com/JuliaApproximation/ApproxFun.jl)
    - [`IntervalRootFinding.jl`](https://github.com/JuliaIntervals/IntervalRootFinding.jl)
    - [`MDBM.jl`](https://github.com/bachrathyd/MDBM.jl)
    - [`Roots.jl`](https://github.com/JuliaMath/Roots.jl)

Each of these uses different techniques, with different advantages and disadvantages.

The challenge exercise for the workshop is: develop a package which integrates all of these disparate packages into a coherent whole!

### Logistics of the workshop

The workshop is based around a series of exercises to be done during the workshop. We will pause to work on the exercises and then I will discuss possible solutions during the workshop.

These techniques are useful for both users and developers; indeed, in Julia the distinction between users and developers is not useful, since it's much easier than in other languages to join the two categories together.

### Outline

We will start by quickly reviewing roots of functions and quickly reviewing one of the standard algorithms, **Newton's algorithm**. We will restrict to finding roots of 1D functions for simplicity.

Newton's algorithm requires the calculation of derivatives, for which several choices of algorithm are available. We will see how to encode the choice of algorithm using dispatch.

Then we will define types which will contain all information about a root-finding problem.

## Roots

Given a function $f: \mathbb{R} \to \mathbb{R}$ (i.e. that accepts a single real number as argument and returns another real number), recall that a **root** or **zero** of the function is a number $x^*$ such that

$$ f(x^*) = 0, $$

i.e. it is a solution of the equation $f(x) = 0$.

In general it is impossible to solve this equation exactly for $x^*$, so we use iterative numerical algorithms instead.

#### Example

Recall that the function $f$ given by $f(x) := x^2 - 2$ has exactly two roots, at $x^*_1 = +\sqrt{2}$ and $x^*_2 = -\sqrt{2}$. Note that it is impossible to represent these values exactly using floating-point arithmetic.

## Newton algorithm

The Newton algorithm for (possibly) finding a root of a nonlinear function $f(x)$ in 1D is the following iteration:

$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)},$$

where $f'$ is the derivative of $f$. We start from an initial guess $x_0$ that can be almost anything (except points for which $f'(x_0) = 0$).

#### Exercise 1

1. Implement the Newton algorithm for a fixed number $n$ of steps in a function `newton`, starting from a given starting point $x_0$.  

    Hint: Which information does the function require?


2. Does your function work with other number types, such as `BigFloat`? What do you need in order to run it with those types? Use it to calculate $\sqrt{2}$. How many decimal places are correct with the standard precision of `BigFloat`?

Want to find roots of some Julia function. How can we define a function in Julia?

In [5]:
function f(x)
    
    @show x   # prints out the value of x
    
    y = x^2 - 2
    
    @show y

    return y   # always return information from a function
end

f (generic function with 1 method)

In [6]:
f2(x) = x^2 - 2

f2 (generic function with 1 method)

In [7]:
f(10)

x = 10
y = 98


98

The notebook automatically shows you the value that was returned.

In [8]:
result = f(10)

x = 10
y = 98


98

In [9]:
result

98

In [10]:
f2(10)

98

In [11]:
function g(x, y)
   
    return (x + y, x - y)
    
end

g (generic function with 1 method)

In [12]:
g(1, 2)

(3, -1)

In [13]:
π

π = 3.1415926535897...

In [14]:
function volume_sphere(r)
    return (4/3) * π * r^3
end

volume_sphere (generic function with 1 method)

In [15]:
volume_sphere(2.0)

33.510321638291124

In [35]:
volume_sphere(2)

33.510321638291124

In [36]:
2^3

8

0.125

-0.125

In [16]:
x = 2

2

In [17]:
x

2

What "kind of thing" is the value 2?  The technical word for "kind of thing" is "type".

In [18]:
typeof(x)

Int64

"64-bit integer"

In [19]:
bitstring(x)

"0000000000000000000000000000000000000000000000000000000000000010"

In [20]:
y = Float64(x)

2.0

In [21]:
typeof(y)

Float64

"64-bit floating-point number" -- approximation of a real number.

3.14 is a "real number"

In [22]:
bitstring(y)

"0100000000000000000000000000000000000000000000000000000000000000"

In [23]:
bitstring(3.0)

"0100000000001000000000000000000000000000000000000000000000000000"

In [24]:
bitstring(3)

"0000000000000000000000000000000000000000000000000000000000000011"

In [25]:
6 / 2

3.0

In [27]:
7 / 2

3.5

In [28]:
div(7, 2)

3

In [29]:
7 ÷ 2   # \div -- integer division, returns an integer (whole number) result

3

In [30]:
7 / 2

3.5

In [31]:
6 / 2

3.0

In [26]:
6

6

In [ ]:
6.0

In [32]:
6 * 2

12

In [33]:
g(x) = (x / 2) * 2

g (generic function with 2 methods)

In [34]:
g(10)

10.0

In [40]:
@code_lowered g(10)

CodeInfo(
1 ─ %1 = x / 2
│   %2 = %1 * 2
└──      return %2
)

In [39]:
@code_warntype g(10)

Body::Float64
1 ─ %1 = (Base.sitofp)(Float64, x)::Float64
│   %2 = (Base.div_float)(%1, 2.0)::Float64
│   %3 = (Base.mul_float)(%2, 2.0)::Float64
└──      return %3


In [41]:
@code_llvm g(10)


;  @ In[33]:1 within `g'
define double @julia_g_12960(i64) {
top:
; ┌ @ int.jl:59 within `/'
; │┌ @ float.jl:271 within `float'
; ││┌ @ float.jl:256 within `Type' @ float.jl:60
     %1 = sitofp i64 %0 to double
; │└└
; │ @ int.jl:59 within `/' @ float.jl:401
   %2 = fmul double %1, 5.000000e-01
; └
; ┌ @ promotion.jl:314 within `*' @ float.jl:399
   %3 = fmul double %2, 2.000000e+00
; └
  ret double %3
}


In [42]:
@code_native g(10)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[33]:1 within `g'
; │┌ @ int.jl:59 within `/'
; ││┌ @ float.jl:271 within `float'
; │││┌ @ float.jl:256 within `Type' @ In[33]:1
	vcvtsi2sdl	%edi, %xmm0, %xmm0
	decl	%eax
	movl	$269143752, %eax        ## imm = 0x100ACEC8
	addl	%eax, (%eax)
	addb	%al, (%eax)
; │└└└
; │┌ @ float.jl:401 within `/'
	vmulsd	(%eax), %xmm0, %xmm0
; │└
; │┌ @ promotion.jl:314 within `*' @ float.jl:399
	vaddsd	%xmm0, %xmm0, %xmm0
; │└
	retl
	nopl	(%eax,%eax)
; └


In [43]:
f(x) = x^2 - 2

f (generic function with 1 method)

In [44]:
df(x) = 2x

df (generic function with 1 method)

## Loops

In [47]:
for i in 1:10
    @show i
    @show i^2
    y = i - 3
    @show y
    
    println()
end

i = 1
i ^ 2 = 1
y = -2

i = 2
i ^ 2 = 4
y = -1

i = 3
i ^ 2 = 9
y = 0

i = 4
i ^ 2 = 16
y = 1

i = 5
i ^ 2 = 25
y = 2

i = 6
i ^ 2 = 36
y = 3

i = 7
i ^ 2 = 49
y = 4

i = 8
i ^ 2 = 64
y = 5

i = 9
i ^ 2 = 81
y = 6

i = 10
i ^ 2 = 100
y = 7



Newton method:

$$x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}$$

$$x_{\text{new}} = x_{\text{current}} - \frac{f(x_{\text{current}})}{f'(x_{\text{current}})}$$

In [49]:
[1, 2, 6]

3-element Array{Int64,1}:
 1
 2
 6

In [52]:
sqrt(2.0)

1.4142135623730951

In [57]:
x0 = 1.0

1.0

x is the current value of x

In [58]:
f(x0)

-1.0

In [59]:
df(x0)

2.0

In [60]:
x_new = x0 - ( f(x0) / df(x0) )

1.5

In [61]:
x1 = x0 - ( f(x0) / df(x0) )

1.5

In [62]:
x2 = x1 - ( f(x1) / df(x1) )

1.4166666666666667

In [63]:
x3 = x2 - ( f(x2) / df(x2) )

1.4142156862745099

In [64]:
x4 = x3 - ( f(x3) / df(x3) )

1.4142135623746899

In [65]:
newton_step(x) = x - ( f(x) / df(x) )

newton_step (generic function with 1 method)

In [66]:
x0 = 1.0

1.0

In [67]:
x1 = newton_step(x0)

1.5

In [68]:
x2 = newton_step(x1)

1.4166666666666667

In [69]:
x0 = 1.0

x = x0

for i in 1:4
    x_new = newton_step(x)
    
    @show x, x_new
    
end

(x, x_new) = (1.0, 1.5)
(x, x_new) = (1.0, 1.5)
(x, x_new) = (1.0, 1.5)
(x, x_new) = (1.0, 1.5)


In [70]:
x0 = 1.0

x = x0

for i in 1:4
    x_new = newton_step(x)
    
    @show x, x_new
    
    x = x_new
    
end

(x, x_new) = (1.0, 1.5)
(x, x_new) = (1.5, 1.4166666666666667)
(x, x_new) = (1.4166666666666667, 1.4142156862745099)
(x, x_new) = (1.4142156862745099, 1.4142135623746899)


In [72]:
x0 = 1.0

x = x0

for i in 1:20
    x_new = newton_step(x)
    
    @show x, x_new
    
    x = x_new
    
end

(x, x_new) = (1.0, 1.5)
(x, x_new) = (1.5, 1.4166666666666667)
(x, x_new) = (1.4166666666666667, 1.4142156862745099)
(x, x_new) = (1.4142156862745099, 1.4142135623746899)
(x, x_new) = (1.4142135623746899, 1.4142135623730951)
(x, x_new) = (1.4142135623730951, 1.414213562373095)
(x, x_new) = (1.414213562373095, 1.4142135623730951)
(x, x_new) = (1.4142135623730951, 1.414213562373095)
(x, x_new) = (1.414213562373095, 1.4142135623730951)
(x, x_new) = (1.4142135623730951, 1.414213562373095)
(x, x_new) = (1.414213562373095, 1.4142135623730951)
(x, x_new) = (1.4142135623730951, 1.414213562373095)
(x, x_new) = (1.414213562373095, 1.4142135623730951)
(x, x_new) = (1.4142135623730951, 1.414213562373095)
(x, x_new) = (1.414213562373095, 1.4142135623730951)
(x, x_new) = (1.4142135623730951, 1.414213562373095)
(x, x_new) = (1.414213562373095, 1.4142135623730951)
(x, x_new) = (1.4142135623730951, 1.414213562373095)
(x, x_new) = (1.414213562373095, 1.4142135623730951)
(x, x_new) = (1.4142135623730951,

Exercise: Use a `while` loop instead to stop the iteration when $f(x)$ is close to $0$.

Stop the loop when $f(x)$ is close to 0:

In [74]:
tolerance = 1e-10  # 10^(-10)

1.0e-10

In [76]:
x0 = 1.0

x = x0

for i in 1:20
    x_new = newton_step(x)
    
    @show i, x_new
    
    x = x_new
    
    if abs(f(x)) < tolerance   # distance of f(x) from 0
        break  # jump out of the loop
    end
    
end

(i, x_new) = (1, 1.5)
(i, x_new) = (2, 1.4166666666666667)
(i, x_new) = (3, 1.4142156862745099)
(i, x_new) = (4, 1.4142135623746899)


In [77]:
x0 = 1000.0

x = x0

for i in 1:20
    x_new = newton_step(x)
    
    @show i, x_new
    
    x = x_new
    
    if abs(f(x)) < tolerance   # distance of f(x) from 0
        break  # jump out of the loop
    end
    
end

(i, x_new) = (1, 500.001)
(i, x_new) = (2, 250.00249999599998)
(i, x_new) = (3, 125.00524995800045)
(i, x_new) = (4, 62.51062464301702)
(i, x_new) = (5, 31.27130960206219)
(i, x_new) = (6, 15.667632994868365)
(i, x_new) = (7, 7.897642347856357)
(i, x_new) = (8, 4.075441240519499)
(i, x_new) = (9, 2.283092824392554)
(i, x_new) = (10, 1.5795487524060154)
(i, x_new) = (11, 1.4228665795786684)
(i, x_new) = (12, 1.4142398735915307)
(i, x_new) = (13, 1.4142135626178485)
(i, x_new) = (14, 1.4142135623730951)


In [79]:
x0 = 1000000.0

x = x0

for i in 1:10000
    x_new = newton_step(x)
    
    @show i, x_new
    
    x = x_new
    
    if abs(f(x)) < tolerance   # distance of f(x) from 0
        break  # jump out of the loop
    end
    
end

(i, x_new) = (1, 500000.000001)
(i, x_new) = (2, 250000.00000250002)
(i, x_new) = (3, 125000.00000525001)
(i, x_new) = (4, 62500.00001062501)
(i, x_new) = (5, 31250.000021312506)
(i, x_new) = (6, 15625.000042656253)
(i, x_new) = (7, 7812.500085328126)
(i, x_new) = (8, 3906.2501706640614)
(i, x_new) = (9, 1953.1253413320196)
(i, x_new) = (10, 976.5631826659203)
(i, x_new) = (11, 488.2826153322443)
(i, x_new) = (12, 244.14335566039554)
(i, x_new) = (13, 122.0757737843854)
(i, x_new) = (14, 61.0460785257085)
(i, x_new) = (15, 30.539420331452767)
(i, x_new) = (16, 15.302454729609611)
(i, x_new) = (17, 7.716576357346835)
(i, x_new) = (18, 3.9878793281276588)
(i, x_new) = (19, 2.244699508512159)
(i, x_new) = (20, 1.5678436817095691)
(i, x_new) = (21, 1.4217405288183735)
(i, x_new) = (22, 1.4142334869735127)
(i, x_new) = (23, 1.41421356251345)
(i, x_new) = (24, 1.414213562373095)


In [91]:
function newton(x0)

    x = x0
    

    for i in 1:10000
        x_new = newton_step(x)

        # @show i, x_new

        x = x_new

        if abs(f(x)) < tolerance   # distance of f(x) from 0
            break  # jump out of the loop
        end

    end
    
    return i, x

end

newton (generic function with 2 methods)

In [84]:
newton   # without parentheses, Julia gives me info about the function

newton (generic function with 1 method)

In [92]:
newton(100)   # need parentheses () to *call* the function

UndefVarError: UndefVarError: i not defined

In [94]:
function newton(x0)

    x = x0
    
    num_steps = 0

    for i in 1:10000
        x_new = x - f(x) / df(x)   # newton_step(x)

        # @show i, x_new

        x = x_new

        if abs(f(x)) < tolerance   # distance of f(x) from 0
            num_steps = i
            break  # jump out of the loop
        end

    end
    
    return num_steps, x

end

newton (generic function with 2 methods)

In [95]:
newton(10.0)

(7, 1.4142135623730954)

In [96]:
newton(100.0)

(10, 1.41421356237384)

What happens if I want to find roots of a different function?

In [97]:
g(x) = x^3 - 2
dg(x) = 3x^2

dg (generic function with 1 method)

In [6]:
function newton(f, df, x0, tolerance)

    x = x0
    
    num_steps = 0

    for i in 1:10000
        x_new = x - f(x) / df(x)   # newton_step(x)

        # @show i, x_new

        x = x_new

        if abs(f(x)) < tolerance   # distance of f(x) from 0
            num_steps = i
            break  # jump out of the loop
        end

    end
    
    return num_steps, x

end

newton (generic function with 1 method)

In [107]:
newton(f, df, 3.0, 1e-10)

(5, 1.4142135623731118)

In [31]:
function newton(f, df, x0, tolerance=1e-10)  # default value

    x = x0
    
    num_steps = -1  # sentinel value

    for i in 1:10000
        x_new = x - f(x) / df(x)   # newton_step(x)

        # @show i, x_new

        x = x_new

        if abs(f(x)) < tolerance   # distance of f(x) from 0
            num_steps = i
            break  # jump out of the loop
        end

    end
    
    return num_steps, x

end

newton (generic function with 2 methods)

In [109]:
newton(f, df, 3.0)

(5, 1.4142135623731118)

In [110]:
newton

newton (generic function with 4 methods)

In [111]:
methods(newton)

# 4 methods for generic function "newton":
[1] newton() in Main at In[80]:3
[2] newton(x0) in Main at In[94]:3
[3] newton(f, df, x0) in Main at In[108]:3
[4] newton(f, df, x0, tolerance) in Main at In[108]:3

In [112]:
function hello(x=1, y=2, z=3)
    @show x, y, z
end

hello (generic function with 4 methods)

In [113]:
methods(hello)

# 4 methods for generic function "hello":
[1] hello() in Main at In[112]:2
[2] hello(x) in Main at In[112]:2
[3] hello(x, y) in Main at In[112]:2
[4] hello(x, y, z) in Main at In[112]:2

In [114]:
function hello(x=1.1)
    @show x, x
end

hello (generic function with 4 methods)

In [115]:
methods(hello)

# 4 methods for generic function "hello":
[1] hello() in Main at In[114]:2
[2] hello(x) in Main at In[114]:2
[3] hello(x, y) in Main at In[112]:2
[4] hello(x, y, z) in Main at In[112]:2

In [102]:
newton(x -> x^4 - 2, x -> 4x^3, 3.0)   # anonymous functions

(8, 1.189207115002721)

In [104]:
2^(1/4)

1.189207115002721

"The function that sends x to x^4 - 2":  `x -> x^4 - 2`

In maths: $x \mapsto x^4 - 2$

In [105]:
newton(3.0, f, df)

MethodError: MethodError: objects of type Float64 are not callable

(1.414213562373095, 10)

In [5]:
sqrt(2)

1.4142135623730951

In [116]:
f(x) = x^2 - 2

f (generic function with 1 method)

In [117]:
newton(f, df, -10.0)

(7, -1.4142135623730954)

In [119]:
g(x) = x^3 - 1
dg(x) = 3x^2

newton(g, dg, 1 + im)

(8, 0.9999999999999994 - 4.556244651765188e-16im)

In [121]:
newton(g, dg, -1 + im)

(5, -0.4999999999999555 + 0.8660254037846933im)

In [ ]:
newton(g)

In [ ]:
function roots(f, df)
    
end

In [123]:
@code_warntype newton(g, dg, 1+im, 1e-10)

Body::Tuple{Int64,Complex{Float64}}
1 ──        goto #31 if not true
2 ┄─ %2   = φ (#30 => %172)::Float64
│    %3   = φ (#30 => %173)::Float64
│    %4   = φ (#30 => %174)::Float64
│    %5   = φ (#30 => %175)::Float64
│    %6   = φ (#30 => %176)::Float64
│    %7   = φ (#30 => %177)::Float64
│    %8   = φ (#30 => %178)::Float64
│    %9   = φ (#30 => %179)::Float64
│    %10  = φ (#30 => %180)::Float64
│    %11  = φ (#30 => %181)::Float64
│    %12  = φ (#30 => %182)::Float64
│    %13  = φ (#30 => %183)::Float64
│    %14  = φ (#30 => %184)::Float64
│    %15  = φ (#30 => %185)::Float64
│    %16  = φ (#30 => %186)::Float64
│    %17  = φ (#30 => %187)::Float64
│    %18  = φ (#30 => %188)::Float64
│    %19  = φ (#30 => %189)::Float64
│    %20  = φ (#30 => %190)::Float64
│    %21  = φ (#30 => %191)::Float64
│    %22  = φ (#30 => %192)::Float64
│    %23  = φ (#30 => %193)::Float64
│    %24  = φ (#1 => _4, #30 => %206)::Union{Complex{Float64}, Complex{Int64}}
│    %25  = φ (#1 => 1, #30 => %229)::

│    %196 = φ (#20 => %154, #22 => %167)::Float64
│    %197 = φ (#20 => %154, #22 => %167)::Float64
│    %198 = φ (#20 => %152, #22 => %163)::Float64
│    %199 = φ (#20 => %154, #22 => %167)::Float64
│    %200 = φ (#20 => %154, #22 => %167)::Float64
│    %201 = φ (#20 => %152, #22 => %163)::Float64
│    %202 = φ (#20 => %152, #22 => %163)::Float64
│    %203 = φ (#20 => %154, #22 => %167)::Float64
│    %204 = φ (#20 => %154, #22 => %167)::Float64
│    %205 = φ (#20 => %152, #22 => %163)::Float64
│    %206 = φ (#20 => %155, #22 => %168)::Complex{Float64}
│    %207 = (Base.mul_float)(%194, %195)::Float64
│    %208 = (Base.mul_float)(%196, %197)::Float64
│    %209 = (Base.sub_float)(%207, %208)::Float64
│    %210 = (Base.mul_float)(%198, %199)::Float64
│    %211 = (Base.mul_float)(%200, %201)::Float64
│    %212 = (Base.add_float)(%210, %211)::Float64
│    %213 = (Base.mul_float)(%209, %202)::Float64
│    %214 = (Base.mul_float)(%212, %203)::Float64
│    %215 = (Base.sub_float)(%213, %214):

In [124]:
newton(f, df, 1+im)

(5, 1.4142135623746899 + 0.0im)

In [127]:
m = @which newton(f, df, 1+im)

newton(f, df, x0) in Main at In[108]:3

In [128]:
typeof(m)

Method

In [129]:
m.specializations

Core.TypeMapEntry(Core.TypeMapEntry(Core.TypeMapEntry(nothing, Tuple{typeof(newton),Function,Function,Float64}, nothing, svec(), 25606, -1, MethodInstance for newton(::Function, ::Function, ::Float64), false, false, false), Tuple{typeof(newton),Function,Function,Complex{Int64}}, nothing, svec(), 25606, -1, MethodInstance for newton(::Function, ::Function, ::Complex{Int64}), false, false, false), Tuple{typeof(newton),typeof(g),typeof(dg),Complex{Int64}}, nothing, svec(), 25616, -1, MethodInstance for newton(::typeof(g), ::typeof(dg), ::Complex{Int64}), true, true, false)

In [130]:
h(x) = x^2 + 1

h (generic function with 1 method)

In [131]:
dh(x) = 2x

dh (generic function with 1 method)

In [135]:
newton(h, dh, 3.0)

(-1, 0.47231766033837386)

## Find many roots

Idea: Run Newton algorithm from many starting points

Real roots: Take many initial values x0 on real line

In [13]:
initial_values = -10:0.1:10

-10.0:0.1:10.0

In [14]:
collect(initial_values)  # turns the range into an array

201-element Array{Float64,1}:
 -10.0
  -9.9
  -9.8
  -9.7
  -9.6
  -9.5
  -9.4
  -9.3
  -9.2
  -9.1
  -9.0
  -8.9
  -8.8
   ⋮  
   8.9
   9.0
   9.1
   9.2
   9.3
   9.4
   9.5
   9.6
   9.7
   9.8
   9.9
  10.0

In [15]:
typeof(initial_values)

StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}

In [8]:
f(x) = x^2 - 2
df(x) = 2x

df (generic function with 1 method)

In [12]:
0.0 / 0.0

NaN

In [11]:
for x0 in initial_values    # makes x0 take each value in initial_values, one by one
    # "foreach" in other languages
    result = newton(f, df, x0)
    @show x0, result
end

(x0, result) = (-10.0, (7, -1.4142135623730954))
(x0, result) = (-9.9, (7, -1.4142135623730954))
(x0, result) = (-9.8, (7, -1.4142135623730951))
(x0, result) = (-9.7, (7, -1.4142135623730951))
(x0, result) = (-9.6, (7, -1.4142135623730951))
(x0, result) = (-9.5, (7, -1.4142135623730951))
(x0, result) = (-9.4, (7, -1.4142135623730951))
(x0, result) = (-9.3, (7, -1.414213562373095))
(x0, result) = (-9.2, (7, -1.4142135623730951))
(x0, result) = (-9.1, (7, -1.4142135623730951))
(x0, result) = (-9.0, (7, -1.4142135623730951))
(x0, result) = (-8.9, (7, -1.4142135623730951))
(x0, result) = (-8.8, (7, -1.4142135623730951))
(x0, result) = (-8.7, (7, -1.4142135623730951))
(x0, result) = (-8.6, (7, -1.4142135623730951))
(x0, result) = (-8.5, (7, -1.4142135623730951))
(x0, result) = (-8.4, (7, -1.414213562373095))
(x0, result) = (-8.3, (7, -1.414213562373095))
(x0, result) = (-8.2, (7, -1.414213562373095))
(x0, result) = (-8.1, (7, -1.4142135623730951))
(x0, result) = (-8.0, (7, -1.41421356237309

In [16]:
for i in 1:length(initial_values)    # makes x0 take each value in initial_values, one by one
    # "foreach" in other languages
    result = newton(f, df, initial_values[i])
    @show initial_values[i], result
end)

LoadError: syntax: extra token ")" after end of expression

In [20]:
for x0 in initial_values    # makes x0 take each value in initial_values, one by one
    # "foreach" in other languages
    result = newton(f, df, x0)
    @show x0, result
end

(x0, result) = (-10.0, (7, -1.4142135623730954))
(x0, result) = (-9.9, (7, -1.4142135623730954))
(x0, result) = (-9.8, (7, -1.4142135623730951))
(x0, result) = (-9.7, (7, -1.4142135623730951))
(x0, result) = (-9.6, (7, -1.4142135623730951))
(x0, result) = (-9.5, (7, -1.4142135623730951))
(x0, result) = (-9.4, (7, -1.4142135623730951))
(x0, result) = (-9.3, (7, -1.414213562373095))
(x0, result) = (-9.2, (7, -1.4142135623730951))
(x0, result) = (-9.1, (7, -1.4142135623730951))
(x0, result) = (-9.0, (7, -1.4142135623730951))
(x0, result) = (-8.9, (7, -1.4142135623730951))
(x0, result) = (-8.8, (7, -1.4142135623730951))
(x0, result) = (-8.7, (7, -1.4142135623730951))
(x0, result) = (-8.6, (7, -1.4142135623730951))
(x0, result) = (-8.5, (7, -1.4142135623730951))
(x0, result) = (-8.4, (7, -1.414213562373095))
(x0, result) = (-8.3, (7, -1.414213562373095))
(x0, result) = (-8.2, (7, -1.414213562373095))
(x0, result) = (-8.1, (7, -1.4142135623730951))
(x0, result) = (-8.0, (7, -1.41421356237309

In [21]:
roots = [ ]

0-element Array{Any,1}

In [22]:
roots = Float64[ ]

0-element Array{Float64,1}

In [23]:
push!(roots, 10)

1-element Array{Float64,1}:
 10.0

In [24]:
roots

1-element Array{Float64,1}:
 10.0

In [25]:
push!(roots, 20)

2-element Array{Float64,1}:
 10.0
 20.0

In [26]:
p = (3, 4)

(3, 4)

In [27]:
a, b = p

(3, 4)

In [28]:
a

3

In [29]:
b

4

In [30]:
roots = Float64[ ] 

for x0 in initial_values    # makes x0 take each value in initial_values, one by one
    # "foreach" in other languages
    (status, result) = newton(f, df, x0)   # newton returns a pair
    
#     if status != -1   # exclude non-roots   # != means "not equal"
#         push!(roots, result)
#     end
    
    if status == -1   # to check equality, use two equals signs
        # nothing
    else
        push!(roots, result)
    end
    
end

In [46]:
x = NaN

NaN

In [47]:
typeof(x)

Float64

In [48]:
Inf - Inf

NaN

In [49]:
x = NaN

NaN

In [50]:
x == NaN

false

In [51]:
isnan(x)

true

In [52]:
!isnan(x)

false

In [53]:
pwd()

"/Users/dpsanders"

roots

In [32]:
unique(roots)

90-element Array{Float64,1}:
 -1.4142135623730954
 -1.4142135623730951
 -1.414213562373095 
 -1.414213562408124 
 -1.4142135623975822
 -1.414213562390034 
 -1.4142135623846848
 -1.4142135623809344
 -1.4142135623783343
 -1.4142135623765528
 -1.4142135623753471
 -1.4142135623745418
 -1.414213562374011 
  ⋮                 
  1.4142135623734455
  1.4142135623736662
  1.414213562374011 
  1.4142135623745418
  1.4142135623753471
  1.4142135623765528
  1.4142135623783343
  1.4142135623809344
  1.4142135623846848
  1.414213562390034 
  1.4142135623975822
  1.414213562408124 

In [35]:
r = roots[1]  # 1st element in the array

-1.4142135623730954

In [37]:
round(r, digits=10)   # keyword

-1.4142135624

In [38]:
roots

200-element Array{Float64,1}:
 -1.4142135623730954
 -1.4142135623730954
 -1.4142135623730951
 -1.4142135623730951
 -1.4142135623730951
 -1.4142135623730951
 -1.4142135623730951
 -1.414213562373095 
 -1.4142135623730951
 -1.4142135623730951
 -1.4142135623730951
 -1.4142135623730951
 -1.4142135623730951
  ⋮                 
  1.4142135623730951
  1.4142135623730951
  1.4142135623730951
  1.4142135623730951
  1.414213562373095 
  1.4142135623730951
  1.4142135623730951
  1.4142135623730951
  1.4142135623730951
  1.4142135623730951
  1.4142135623730954
  1.4142135623730954

How can we round all the elements in the array?

In [39]:
rounded = Float64[ ] 

for root in roots
    push!(rounded, round(root, digits=10) )
end

In [40]:
rounded

200-element Array{Float64,1}:
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
  ⋮           
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624

In [41]:
unique(rounded)

2-element Array{Float64,1}:
 -1.4142135624
  1.4142135624

Can we write less code to apply the `round` function to each element of the array?

In [42]:
[ round(x, digits=10) for x in roots ]  # array comprehension

200-element Array{Float64,1}:
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
  ⋮           
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624

Even shorter syntax: using broadcasting, `.`

In [43]:
round(roots, digits=10)

MethodError: MethodError: no method matching round(::Array{Float64,1}; digits=10)
Closest candidates are:
  round(!Matched::Type{BigInt}, !Matched::BigFloat) at mpfr.jl:315 got unsupported keyword argument "digits"
  round(!Matched::Float64, !Matched::RoundingMode{:Nearest}) at float.jl:370 got unsupported keyword argument "digits"
  round(!Matched::Float64, !Matched::RoundingMode{:Up}) at float.jl:368 got unsupported keyword argument "digits"
  ...

In [44]:
round.(roots, digits=10)

200-element Array{Float64,1}:
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
 -1.4142135624
  ⋮           
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624
  1.4142135624

In [55]:
pwd()

"/Users/dpsanders"

Exercise: Write a function that finds many roots of a given function.

In [45]:
roots(f, df)

MethodError: MethodError: objects of type Array{Float64,1} are not callable
Use square brackets [] for indexing an Array.

## Calculating derivatives

The Newton algorithm requires us to specify the derivative of a function. If $f$ is a complicated function, we certainly don't want to do that by hand.

One standard solution is to use a *finite-difference approximation*:

$$f'(a) \simeq \frac{f(a + h) - f(a - h)}{2h}.$$

#### Exercise 2

1. Implement a function `finite_difference` with a default value $h = 0.001$ to calculate $f'(a)$ at a given point $a$.


2. Use an anonymous function to make a method of `finite_difference` that calculates the *function* $f'$, i.e. the function that takes $a$ to $f'(a)$.


3. Implement a version of `newton` that does not take the derivative as argument, i.e. `newton(f, x0)`, and uses `finite_difference` to calculate the derivative. This version of `newton` should **re-use** the previous version by defining the function `df` and calling that version.

In [2]:
finite_difference(f, a, h=0.001) = ( f(a + h) - f(a - h) ) / (2h)

finite_difference (generic function with 2 methods)

In [3]:
methods(finite_difference)

# 2 methods for generic function "finite_difference":
[1] finite_difference(f, a) in Main at In[2]:1
[2] finite_difference(f, a, h) in Main at In[2]:1

In [4]:
finite_difference(f, a; h=0.001) = ( f(a + h) - f(a - h) ) / (2h)
# arguments after ; are keyword arguments

finite_difference (generic function with 2 methods)

In [12]:
f(x) = x^3 - 2

f (generic function with 1 method)

In [16]:
a = 2.0
finite_difference(f, a, h=0.01)

12.000099999999847

In [17]:
df(x) = 3x^2

df (generic function with 1 method)

In [18]:
df(a)

12.0

In [20]:
abs(df(a) - finite_difference(f, a, h=0.01))

9.999999984700025e-5

In [21]:
abs(df(a) - finite_difference(f, a, h=0.1))

0.010000000000008669

In [22]:
finite_difference(f)  # should give me a function

MethodError: MethodError: no method matching finite_difference(::typeof(f))
Closest candidates are:
  finite_difference(::Any, !Matched::Any; h) at In[4]:1
  finite_difference(::Any, !Matched::Any, !Matched::Any) at In[2]:1

In [36]:
# finite_difference(f, a)  # returns a number, approx of f'(a)

In [39]:
finite_difference(f, 3.0)

27.000000999995777

In [24]:
finite_difference(f) = a -> finite_difference(f, a)

finite_difference (generic function with 3 methods)

In [ ]:
∂(f) = a -> finite_difference(f, a)

In [28]:
deriv_f = finite_difference(f)

#8 (generic function with 1 method)

In [29]:
deriv_f(3.0)

27.000000999995777

In [30]:
newton

UndefVarError: UndefVarError: newton not defined

In [ ]:
x1 = x0 - f(x0) / df(x0)

In [34]:
function newton(f, x0)
    # df = finite_difference(f)
    
    # df = x -> finite_difference(f, x)
    
    df(x) = finite_difference(f, x)
    
    return newton(f, df, x0)
    
end

newton (generic function with 3 methods)

In [35]:
newton(f, 3.0)

(6, 1.2599210498993205)

In [33]:
methods(newton)

# 3 methods for generic function "newton":
[1] newton(f, x0) in Main at In[32]:4
[2] newton(f, df, x0) in Main at In[31]:3
[3] newton(f, df, x0, tolerance) in Main at In[31]:3

### Algorithmic differentiation

An alternative way to calculate derivatives is by using [**algorithmic differentiation**](https://en.wikipedia.org/wiki/Automatic_differentiation) (also called **automatic differentiation** or **computational differentiation**). This gives exact results (up to rounding error).


We will implement this algorithm in the next notebook, but for now let's just use the implementation in the excellent [`ForwardDiff.jl` package](https://github.com/JuliaDiff/ForwardDiff.jl).


#### Exercise 3

1. Install `ForwardDiff.jl` if necessary.


2. Import it.


3. Define a function `forwarddiff(f, x)` that uses the `ForwardDiff.derivative` function to calculate a derivative.

In [40]:
using Pkg

Pkg.add("ForwardDiff")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %14.4 %>                            ]  28.7 % [==================>                      ]  43.1 %]  57.3 % [=============================>           ]  71.6 %>   ]  90.5 % Resolving package versions...
 Installed LoweredCodeUtils ─ v0.3.7
 Installed Distances ──────── v0.8.1
 Installed HDF5 ───────────── v0.12.1
 Installed StringDistances ── v0.4.0
 Installed DelayDiffEq ────── v5.14.0
 Installed Zygote ─────────── v0.3.4
 Installed AxisArrays ─────── v0.3.2
 Installed Roots ──────────── v0.8.3
 Installed Mocking ────────── v0.6.0
 Installed EzXML ──────────── v0.9.4
 Installed ZygoteRules ────── v0.1.0
 Installed IRTools ────────── v0.2.3
 Installed FillArrays ─────── v0.6.4
 Installed Sundials ───────── v3.6.2
 Installed Highlights ─────── v0.4.1
  Updating `~/.julia/environments/v1.1/Project.toml`
 

In [41]:
using ForwardDiff

In [42]:
?ForwardDiff.derivative

```
ForwardDiff.derivative(f, x::Real)
```

Return `df/dx` evaluated at `x`, assuming `f` is called as `f(x)`.

This method assumes that `isa(f(x), Union{Real,AbstractArray})`.

---

```
ForwardDiff.derivative(f!, y::AbstractArray, x::Real, cfg::DerivativeConfig = DerivativeConfig(f!, y, x), check=Val{true}())
```

Return `df!/dx` evaluated at `x`, assuming `f!` is called as `f!(y, x)` where the result is stored in `y`.

Set `check` to `Val{false}()` to disable tag checking. This can lead to perturbation confusion, so should be used with care.


In [44]:
f(x) = x^2 - 2

ForwardDiff.derivative(f, 3.0)

6.0

In [45]:
ForwardDiff.derivative(f, 3.0 + 4im)

MethodError: MethodError: no method matching derivative(::typeof(f), ::Complex{Float64})
Closest candidates are:
  derivative(::F, !Matched::R<:Real) where {F, R<:Real} at /Users/dpsanders/.julia/packages/ForwardDiff/N0wMF/src/derivative.jl:13
  derivative(::Any, !Matched::AbstractArray, !Matched::Real) at /Users/dpsanders/.julia/packages/ForwardDiff/N0wMF/src/derivative.jl:27
  derivative(::Any, !Matched::AbstractArray, !Matched::Real, !Matched::ForwardDiff.DerivativeConfig{T,D} where D) where T at /Users/dpsanders/.julia/packages/ForwardDiff/N0wMF/src/derivative.jl:27
  ...

In [46]:
forwarddiff(f, x) = ForwardDiff.derivative(f, x)

forwarddiff (generic function with 1 method)

In [47]:
a = 3.0
finite_difference(f, a)

5.999999999999339

In [48]:
forwarddiff(f, a)

6.0

In [49]:
derivative_algorithm = finite_difference

finite_difference (generic function with 3 methods)

In [50]:
derivative_algorithm

finite_difference (generic function with 3 methods)

In [54]:
x = 3

3

In [51]:
derivative_algorithm = forwarddiff

forwarddiff (generic function with 1 method)

In [52]:
derivative_algorithm(f, a)

6.0

In [53]:
derivative_algorithm = finite_difference
derivative_algorithm(f, a)

5.999999999999339

### Choosing between algorithms

We now have two different algorithms available to calculate derivatives. This kind of situation is common in scientific computing; for example, the [`DifferentialEquations.jl`](http://docs.juliadiffeq.org/latest/) ecosystem has some 300 algorithms for solving differential equations. One of the techniques we will learn is how to easily be able to specify different algorithms.

#### Exercise 4

In [55]:
methods(newton)

# 3 methods for generic function "newton":
[1] newton(f, x0) in Main at In[34]:6
[2] newton(f, df, x0) in Main at In[31]:3
[3] newton(f, df, x0, tolerance) in Main at In[31]:3

1. Make a version of the Newton algorithm that takes an argument which is the method (algorithm) to use to calculate the derivative, given as a function. 
The new method should have the signature `newton(f, x0, n, derivative_algorithm)`.
(The **signature** of a function means the collection of arguments that it takes.)

In [ ]:
function newton(f, x0, n, derivative_algorithm)   # WILL OVERWRITE

end

## A first taste of multiple dispatch

In the above, we ended up with a complicated signature. Maybe we would like something simpler, such as 
```
newton(f, x0, derivative)
```
where `derivative` is the derivative method to use (finite differencing or `forwarddiff`).

The problem is that we already have a method for `newton` that takes three arguments, namely `newton(f, df, x0)`. If we define this new method, we will *overwrite* (destroy) that method, since Julia cannot distinguish between the signature `(f, x0, derivative)` and `(f, df, x0)` -- they are both simply three arguments with different names.

There is a way to distinguish them, however: we (as humans) know intuitively that the first method, `(f, df, x0)`, should take arguments of types `(function, function, number)`, whereas `(f, x0, derivative)` should take `(function, number, function)`. So far, however, we have not told Julia this, since although we recognise by eye that `f` is a function and `x0` is a number, for Julia it is quite possible for `f` to be a number and `x0` to be a function!

So what we need is a mechanism to *specify* to Julia which *type* of arguments each version of `newton` takes, in which order.

### Type annotations

When we need to specify types in Julia, we use a *type annotation*, written `::T`, where `T` is the type.

For example, let's define a function `rounded_square` of one argument, `x`, that calculates a rounded-down square. If `x` is an integer then it should just return `x^2`; but if `x` is a float, it should do a more complicated operation. We can write two *methods* for `rounded_square` as follows:

In [56]:
rounded_square(x) = x^2

rounded_square (generic function with 1 method)

In [57]:
rounded_square(3)

9

In [58]:
rounded_square(3.1)   # wrong: does  not give me an integer

9.610000000000001

In [59]:
methods(rounded_square)

# 1 method for generic function "rounded_square":
[1] rounded_square(x) in Main at In[56]:1

If `x` is of type `Float64`, I want to do something special:

In [60]:
function rounded_square(x)
    if typeof(x) == Float64   #  x isa Float64
        return floor(x^2)    # floor rounds down
    
    else 
        return x^2
    end

end

rounded_square (generic function with 1 method)

In [61]:
rounded_square(3)

9

In [62]:
rounded_square(3.0)

9.0

In [63]:
rounded_square(3.1)

9.0

In Julia, *don't* write it like this. Instead:

In [64]:
rounded_square(x) = x^2   # rounded_square(x::Any)   # generic fallback -- works for any case

rounded_square (generic function with 1 method)

Make a special version (method) that is only for when `x` is of type `Float64`:

In [80]:
rounded_square("hello")   # will do whatever "hello"^2  does

"hellohello"

In [82]:
"hello" * "goodbye"

"hellogoodbye"

In [83]:
"hello"^10

"hellohellohellohellohellohellohellohellohellohello"

In [81]:
sqrt("hello")

MethodError: MethodError: no method matching sqrt(::String)
Closest candidates are:
  sqrt(!Matched::Float16) at math.jl:1018
  sqrt(!Matched::Complex{Float16}) at math.jl:1019
  sqrt(!Matched::Missing) at math.jl:1070
  ...

In [66]:
rounded_square(x::Float64) = floor(x^2)   # ::Float64 is a *type annotation*

rounded_square (generic function with 2 methods)

In [68]:
rounded_square(3)

9

In [69]:
@which rounded_square(3)

rounded_square(x) in Main at In[64]:1

In [70]:
rounded_square(3.1)

9.0

In [71]:
@which rounded_square(3.1)

rounded_square(x::Float64) in Main at In[66]:1

In [72]:
a = 3
rounded_square(a)

9

In [73]:
@which rounded_square(a)

rounded_square(x) in Main at In[64]:1

In [74]:
a = 3.1
rounded_square(a)

9.0

In [75]:
@which rounded_square(a)

rounded_square(x::Float64) in Main at In[66]:1

In [67]:
methods(rounded_square)

# 2 methods for generic function "rounded_square":
[1] rounded_square(x::Float64) in Main at In[66]:1
[2] rounded_square(x) in Main at In[64]:1

In [76]:
abc(x::Float64) = x^2

abc (generic function with 1 method)

In [77]:
abc(3)

MethodError: MethodError: no method matching abc(::Int64)
Closest candidates are:
  abc(!Matched::Float64) at In[76]:1

In [78]:
abc(x) = abc(Float64(x))

abc (generic function with 2 methods)

In [79]:
abc(3)

9.0

The process of choosing *which method to call* is called "dispatch"

In [84]:
rounded_square(3)

rounded_square(3.1)

9.0

But:

In [92]:
x = big"1.1"

1.100000000000000000000000000000000000000000000000000000000000000000000000000003

In [93]:
typeof(x)

BigFloat

In [89]:
rounded_square(big"1.1")

1.210000000000000000000000000000000000000000000000000000000000000000000000000012

In [94]:
methods(rounded_square)

# 2 methods for generic function "rounded_square":
[1] rounded_square(x::Float64) in Main at In[66]:1
[2] rounded_square(x) in Main at In[64]:1

We see that we restricted the second method too much: really we would like to allow any real number:

In [95]:
rounded_square(x::Real) = floor(x^2)

rounded_square (generic function with 3 methods)

In [96]:
methods(rounded_square)

# 3 methods for generic function "rounded_square":
[1] rounded_square(x::Float64) in Main at In[66]:1
[2] rounded_square(x::Real) in Main at In[95]:1
[3] rounded_square(x) in Main at In[64]:1

In [86]:
π isa Real

true

In [87]:
typeof(π)

Irrational{:π}

In [88]:
rounded_square(π)

9.869604401089358

In [10]:
rounded_square(big"3.1")

9.0

In [11]:
rounded_square(π)

9.0

If we later discover other cases that we would like to be covered, we can *add new methods* to the function, even for new kinds of types that we define (although if they are subtypes of `Real` then they *are already covered*!)

### Multiple dispatch

Julia checks the types of *all arguments of a function* and chooses a method that matches all of them simultaneously. This is known as **multiple dispatch**. ("Dispatch" is the act of choosing which version of a function to call.)

Although this does not necessarily sound like a complicated idea, it is one of the key things that differentiates Julia from most other programming languages, and it has led to many interesting developments; I highly recommend [Stefan Karpinski's talk from JuliaCon 2019](https://www.youtube.com/watch?v=kc9HwsxE1OY).

In [97]:
function rounded_add(x, y)
    x + y
end

rounded_add (generic function with 1 method)

In [98]:
function rounded_add(x::Float64, y::Float64)
    floor(x + y)
end

rounded_add (generic function with 2 methods)

In [99]:
rounded_add(3, 4)

7

In [100]:
rounded_add(3.1, 4.2)

7.0

In [101]:
function rounded_add(x::Float64, y)
    floor(x + y)
end

rounded_add (generic function with 3 methods)

In [102]:
methods(rounded_add)

# 3 methods for generic function "rounded_add":
[1] rounded_add(x::Float64, y::Float64) in Main at In[98]:2
[2] rounded_add(x::Float64, y) in Main at In[101]:2
[3] rounded_add(x, y) in Main at In[97]:2

In [104]:
@which rounded_add(3.1, 4)

rounded_add(x::Float64, y) in Main at In[101]:2

In [105]:
x = 3.1
y = 4

4

In [106]:
x + y

7.1

In [107]:
@which x + y

+(x::Number, y::Number) in Base at promotion.jl:313

In [108]:
+

+ (generic function with 190 methods)

In [109]:
methods(+)

# 190 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:96
[12] +(::Missing) in Base at missing.jl:83
[13] +(::Missing, ::Number) in Base at missing.jl:97
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:406
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:434
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:433
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:432
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:403
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:440
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:446
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:556
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:549
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:543
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:402
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:371
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:378
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:386
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:394
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:343
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:341
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:371
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:24
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:76
[43]

In [110]:
@which (1 + 2im) + (3 + 4im)

+(z::Complex, w::Complex) in Base at complex.jl:266

In [116]:
@edit (1 + 2im) + 3

In [112]:
+(promote(x, y)...)

(3.1, 4.0)

In [114]:
@which +(3.1, 4.0)

+(x::Float64, y::Float64) in Base at float.jl:395

In [115]:
+(1, 2, 3, 4, 6, 7, 7, 8)

38

Let's return to the `newton` example. 

#### Exercise 5

1. Write a method for `newton` that takes as arguments `f`, `x0` and a `derivative` method by annotating the `derivative` method as being of type `Function`. 


2. Alternatively, annotate the method which takes `f`, `df` and `x0` by annotating `x0` as being `Real`.

In [117]:
function newton(f, x0, n, derivative::Function)
    df = x -> derivative(f, x)
    
    return newton(f, df, x0, n)
end

newton (generic function with 4 methods)

In [118]:
methods(newton)

# 4 methods for generic function "newton":
[1] newton(f, x0) in Main at In[34]:6
[2] newton(f, df, x0) in Main at In[31]:3
[3] newton(f, x0, n, derivative::Function) in Main at In[117]:2
[4] newton(f, df, x0, tolerance) in Main at In[31]:3

In [119]:
methods(+)

# 190 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:96
[12] +(::Missing) in Base at missing.jl:83
[13] +(::Missing, ::Number) in Base at missing.jl:97
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:406
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:434
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:433
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:432
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:403
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:440
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:446
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:556
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:549
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:543
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:402
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:371
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:378
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:386
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:394
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:343
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:341
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:371
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:24
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:76
[43]

In [120]:
3.1 + "hello"

MethodError: MethodError: no method matching +(::Float64, ::String)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502
  +(::Float64, !Matched::Float64) at float.jl:395
  +(::AbstractFloat, !Matched::Bool) at bool.jl:114
  ...

In [121]:
using SparseArrays

In [122]:
?sprand

search: sprand sprandn StepRange StepRangeLen



```
sprand([rng],[type],m,[n],p::AbstractFloat,[rfn])
```

Create a random length `m` sparse vector or `m` by `n` sparse matrix, in which the probability of any element being nonzero is independently given by `p` (and hence the mean density of nonzeros is also exactly `p`). Nonzero values are sampled from the distribution specified by `rfn` and have the type `type`. The uniform distribution is used in case `rfn` is not specified. The optional `rng` argument specifies a random number generator, see [Random Numbers](@ref).

# Examples

```jldoctest; setup = :(using Random; Random.seed!(1234))
julia> sprand(Bool, 2, 2, 0.5)
2×2 SparseMatrixCSC{Bool,Int64} with 2 stored entries:
  [1, 1]  =  true
  [2, 1]  =  true

julia> sprand(Float64, 3, 0.75)
3-element SparseVector{Float64,Int64} with 1 stored entry:
  [3]  =  0.298614
```


In [125]:
M = sprand(10, 10, 0.1)

10×10 SparseMatrixCSC{Float64,Int64} with 9 stored entries:
  [6 ,  1]  =  0.0224559
  [9 ,  1]  =  0.824478
  [8 ,  2]  =  0.102
  [10,  2]  =  0.913097
  [4 ,  6]  =  0.124249
  [8 ,  6]  =  0.181123
  [1 ,  7]  =  0.413704
  [10,  7]  =  0.508078
  [1 ,  9]  =  0.125019

In [126]:
Matrix(M)

10×10 Array{Float64,2}:
 0.0        0.0       0.0  0.0  0.0  0.0       0.413704  0.0  0.125019  0.0
 0.0        0.0       0.0  0.0  0.0  0.0       0.0       0.0  0.0       0.0
 0.0        0.0       0.0  0.0  0.0  0.0       0.0       0.0  0.0       0.0
 0.0        0.0       0.0  0.0  0.0  0.124249  0.0       0.0  0.0       0.0
 0.0        0.0       0.0  0.0  0.0  0.0       0.0       0.0  0.0       0.0
 0.0224559  0.0       0.0  0.0  0.0  0.0       0.0       0.0  0.0       0.0
 0.0        0.0       0.0  0.0  0.0  0.0       0.0       0.0  0.0       0.0
 0.0        0.102     0.0  0.0  0.0  0.181123  0.0       0.0  0.0       0.0
 0.824478   0.0       0.0  0.0  0.0  0.0       0.0       0.0  0.0       0.0
 0.0        0.913097  0.0  0.0  0.0  0.0       0.508078  0.0  0.0       0.0

In [130]:
b = rand(10)

10-element Array{Float64,1}:
 0.33861597750525796 
 0.038008212996762936
 0.8233393941910503  
 0.6280620052594745  
 0.08694730031071618 
 0.2731270937687318  
 0.7285994223703371  
 0.3925564760360085  
 0.9914436473748665  
 0.7325684376206896  

Solve $Mx = b$ -- solution is $x = M^{-1} b$

In [134]:
using LinearAlgebra

In [137]:
M = sprand(10, 10, 0.1)
M2 = I - 0.5*M

10×10 SparseMatrixCSC{Float64,Int64} with 20 stored entries:
  [1 ,  1]  =  1.0
  [2 ,  2]  =  1.0
  [4 ,  2]  =  -0.122783
  [3 ,  3]  =  1.0
  [8 ,  3]  =  -0.0183838
  [10,  3]  =  -0.422903
  [4 ,  4]  =  1.0
  [5 ,  5]  =  1.0
  [7 ,  5]  =  -0.386272
  [1 ,  6]  =  -0.351601
  [4 ,  6]  =  -0.470127
  [5 ,  6]  =  -0.170826
  [6 ,  6]  =  1.0
  [2 ,  7]  =  -0.477744
  [7 ,  7]  =  1.0
  [10,  7]  =  -0.461793
  [8 ,  8]  =  1.0
  [7 ,  9]  =  -0.445729
  [9 ,  9]  =  1.0
  [10, 10]  =  1.0

In [138]:
M2 \ b

10-element Array{Float64,1}:
 0.434647736960265 
 0.621869562008765 
 0.8233393941910503
 0.8328213940827226
 0.1336045053935476
 0.2731270937687318
 1.2221219836096453
 0.4076925543748359
 0.9914436473748665
 1.645128579183626 

In [139]:
@which M2 \ b

\(A::SparseMatrixCSC, B::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T) in SparseArrays at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/SparseArrays/src/linalg.jl:1278

In [141]:
B = sprand(10, 10, 0.1)

10×10 SparseMatrixCSC{Float64,Int64} with 8 stored entries:
  [4 ,  1]  =  0.499127
  [5 ,  1]  =  0.914652
  [9 ,  4]  =  0.0736308
  [3 ,  5]  =  0.170442
  [7 ,  5]  =  0.101829
  [10,  8]  =  0.251637
  [2 , 10]  =  0.847312
  [10, 10]  =  0.559518

In [143]:
M2 \ B

MethodError: MethodError: no method matching ldiv!(::SuiteSparse.UMFPACK.UmfpackLU{Float64,Int64}, ::SparseMatrixCSC{Float64,Int64})
Closest candidates are:
  ldiv!(!Matched::LU{T,Tridiagonal{T,V}}, ::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T) where {T, V} at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/lu.jl:523
  ldiv!(!Matched::Transpose{#s623,#s622} where #s622<:LU{T,Tridiagonal{T,V}} where #s623, ::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T) where {T, V} at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/lu.jl:555
  ldiv!(!Matched::Adjoint{#s623,LU{T,Tridiagonal{T,V}}} where #s623, ::Union{AbstractArray{T,1}, AbstractArray{T,2}} where T) where {T, V} at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/lu.jl:592
  ...